In [1]:
!pip install shapash

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 917.8/917.8 KB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 575.9/575.9 KB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 KB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 59.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 642.7/642.7 KB 35.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4 KB 4.5 MB/s eta 0:00:00
  Created wheel for dash-renderer: filename=dash_renderer-1.8.3-py3-none-any.whl size=1013943 sha256=119d5a8f26ffe21b297f0e1e416f8e0c34a11c397ef4f546a9ac8fab0406ceec
  Stored in directory: /root/.cache/pip/wheels/82/4d/8c/0764de014b25313cf54199f212bad4fb4b2

In [2]:
import pandas as pd
from category_encoders import OrdinalEncoder
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv('/content/drive/MyDrive/Boston.csv', index_col = 0)

In [4]:
df

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
1,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
2,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
3,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
4,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
5,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502,0.06263,0.0,11.93,0,0.573,6.593,69.1,2.4786,1,273,21.0,391.99,9.67,22.4
503,0.04527,0.0,11.93,0,0.573,6.120,76.7,2.2875,1,273,21.0,396.90,9.08,20.6
504,0.06076,0.0,11.93,0,0.573,6.976,91.0,2.1675,1,273,21.0,396.90,5.64,23.9
505,0.10959,0.0,11.93,0,0.573,6.794,89.3,2.3889,1,273,21.0,393.45,6.48,22.0


In [5]:
Xtrain, Xtest, ytrain, ytest = train_test_split(df.iloc[:,:-1], df.iloc[:,-1], train_size=0.75, random_state=1)

In [6]:
regressor = LGBMRegressor(n_estimators=200).fit(Xtrain,ytrain)

In [7]:
y_pred = pd.DataFrame(regressor.predict(Xtest),columns=['pred'],index=Xtest.index)

In [8]:
y_pred

,pred
308,28.271892
344,28.088107
48,19.143672
68,19.949254
363,20.568768
...,...
42,30.125382
362,17.175481
290,24.108724
499,20.328525


In [9]:
# Shapash
from shapash import SmartExplainer

In [10]:
xpl = SmartExplainer(
    model=regressor
    # preprocessing=encoder, # Optional: compile step can use inverse_transform method
    # features_dict=house_dict  # Optional parameter, dict specifies label for features name, {feature:description}
)

In [11]:
xpl.compile(x=Xtest,
 y_pred=y_pred,
 y_target=ytest, # Optional: allows to display True Values vs Predicted Values
 )

In [12]:
xpl.plot.features_importance()

In [13]:
subset = [308,344] # subset index must be in test data
xpl.plot.features_importance(selection=subset)

In [14]:
xpl.plot.contribution_plot("dis")

In [15]:
xpl.plot.local_plot(index=308)

In [16]:
xpl.plot.local_plot(index=344)

In [17]:
xpl.plot.local_plot(index=226)

In [18]:
summary_df = xpl.to_pandas(max_contrib=3)

In [19]:
summary_df

,pred,feature_1,value_1,contribution_1,feature_2,value_2,contribution_2,feature_3,value_3,contribution_3
308,28.271892,lstat,7.53,2.916799,indus,2.18,1.245002,tax,222.0,0.928253
344,28.088107,lstat,7.18,3.131517,rm,6.696,1.14436,ptratio,17.6,0.868808
48,19.143672,lstat,18.8,-2.759395,rm,6.03,-0.944761,crim,0.22927,0.616783
68,19.949254,rm,5.878,-3.522831,lstat,8.1,2.074237,age,21.4,1.12791
363,20.568768,rm,5.362,-1.687492,lstat,10.19,-1.421035,dis,2.1036,1.226687
...,...,...,...,...,...,...,...,...,...,...
42,30.125382,lstat,4.84,6.670954,age,2.9,1.177165,indus,6.91,-0.410887
362,17.175481,lstat,14.19,-2.545545,black,350.65,-1.731845,crim,3.83684,1.039274
290,24.108724,lstat,9.51,3.360089,dis,7.3172,-1.364384,rm,6.565,-0.825537
499,20.328525,lstat,12.92,-1.705392,rm,6.019,-0.897026,dis,2.4091,0.430913


In [20]:
summary_df.iloc[5:50,]

,pred,feature_1,value_1,contribution_1,feature_2,value_2,contribution_2,feature_3,value_3,contribution_3
133,21.097585,nox,0.624,1.56705,lstat,11.12,-1.430232,ptratio,21.2,-1.354986
293,26.183332,lstat,4.7,5.796751,rm,6.63,-1.858748,age,23.4,1.246792
32,17.694382,ptratio,21.0,-1.6943,lstat,13.04,-1.200755,age,100.0,-1.012136
219,22.246191,lstat,17.92,-3.401925,ptratio,16.4,1.564144,rm,5.951,-1.38474
91,24.141333,lstat,8.81,2.797498,rm,6.417,-1.323975,dis,3.0923,1.028223
482,25.864549,lstat,7.74,2.641534,ptratio,20.2,-1.093226,age,74.9,-0.838712
345,28.705419,lstat,4.61,6.411797,dis,6.4654,-1.109788,age,28.1,0.835504
120,21.088360,lstat,13.61,-1.897036,rm,5.731,-1.765792,dis,2.7592,1.114455
67,20.596263,rm,5.787,-2.405791,ptratio,16.1,1.015995,dis,6.6115,-0.66163
313,20.387718,lstat,11.72,-1.298235,rm,6.023,-1.035742,dis,2.834,0.733302


In [21]:

xpl.plot.top_interactions_plot()

In [22]:
# instance 205 both shapley value high when interaction value high
summary_df.loc[205,]

pred              48.189388
feature_1             lstat
value_1                2.88
contribution_1    10.904051
feature_2                rm
value_2               8.034
contribution_2     9.837803
feature_3           ptratio
value_3                14.7
contribution_3     2.364057
Name: 205, dtype: object

In [23]:
xpl.plot.local_plot(index=205)

In [24]:
# You can save the SmartExplainer Object in a pickle file to make new plots later or launch the WebApp again
xpl.save('./xpl.pkl')

In [24]:
# causal effect

In [64]:
y_pred.loc[205,]

pred    48.189388
Name: 205, dtype: float64

In [31]:
Xtest.loc[205,]

crim         0.02009
zn          95.00000
indus        2.68000
chas         0.00000
nox          0.41610
rm           8.03400
age         31.90000
dis          5.11800
rad          4.00000
tax        224.00000
ptratio     14.70000
black      390.55000
lstat        2.88000
Name: 205, dtype: float64

In [84]:
Xtest.columns

Index(['crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'dis', 'rad', 'tax',
       'ptratio', 'black', 'lstat'],
      dtype='object')

In [95]:
index = 226
causal_list = []
for column_name in Xtest.columns:
    
    col_value = Xtest[column_name].unique()
    length = len(col_value)
    df = Xtest.loc[(index,308),].append([Xtest.loc[index,]]*(length-1)).drop(308)
    df[column_name] = col_value
    p = regressor.predict(df)  # new prediction
    
    op = y_pred.loc[index,]  # original prediction
    causal = 0
    for v in p:
      causal += abs(v-op)
  
    print(column_name, causal)
    causal_list.append(causal.values)
print(causal_list)

crim pred    37.864191
Name: 226, dtype: float64
zn pred    5.799318
Name: 226, dtype: float64
indus pred    10.942354
Name: 226, dtype: float64
chas pred    0.201766
Name: 226, dtype: float64
nox pred    22.599106
Name: 226, dtype: float64
rm pred    1263.209004
Name: 226, dtype: float64
age pred    409.935478
Name: 226, dtype: float64
dis pred    121.615765
Name: 226, dtype: float64
rad pred    1.908931
Name: 226, dtype: float64
tax pred    17.582113
Name: 226, dtype: float64
ptratio pred    46.652989
Name: 226, dtype: float64
black pred    43.96907
Name: 226, dtype: float64
lstat pred    1020.924022
Name: 226, dtype: float64
[array([37.86419097]), array([5.79931757]), array([10.9423542]), array([0.20176557]), array([22.59910567]), array([1263.20900376]), array([409.93547813]), array([121.61576486]), array([1.90893117]), array([17.58211317]), array([46.65298867]), array([43.96907]), array([1020.92402187])]
